In [ ]:
!pip install yahoo_fin
!pip install yfinance

In [2]:
import math
import time
import pandas as pd
from datetime import date
from scipy.stats import norm
from yahoo_fin import options
import yfinance as yf

# **Overview:**

This script will allow us to obtain options chains provided by Yahoo Finance. We will go through the option chains, and then apply the Black Scholes Model ("BSM") to calculate the theoretical values and the Greeks. 

**Advantages to BSM:**
* It is widely used
* It's simple to apply

**Disadvantages to BSM:** 
* Was modeled for European-style options; early-exercise is not allowed
* Assumes price to follow a random walk according to a brownian motion with a constant drift. <br>(I have created a separate notebook to explore this topic. [<Link\>](https://github.com/kevinhhl/portfolio-management-tools/blob/main/Monte_Carlo_Simulation_Random_Walk.ipynb))
* Ignores dividends
* Taxes and transaction costs are ignored
* Inputs can be **subjective**, especially for expected volatility

##**Understanding the Model:**

Derivation of the BSM is complex. Through research [1], I'm using my own words to summarize how I understand it from a practical perspective.

The premise of the Black-Scholes Model contains two components. It can be described as taking the difference of: <br>
* (a) the expected value of a stock in the event that it reaches above/below exercise price on the date of expiration for call/put options, respectively, and 
* (b) the expected payout at the exercise price

<br>

These components are expressed as:
<br>\begin{equation}
TV_{call}=se^{rt}N(d_{1})-xN(d_{2})
\end{equation}
<br>\begin{equation}
TV_{put}=xe^{-rt}N(-d_{2})-sN(-d_{1})
\end{equation}

**Where**:
* TV is the theoretical value
* s is the spot price at the current moment
* x is the exercise price of the option
* t is the time till maturity in no. of years
* r is the risk free rate per annum
* σ is the annualized volatility
* N(d) is the CDF of a normal distribution.
<br>
<br>\begin{equation}
d_{1} =\frac{ln(\frac{S}{X})+(r+\frac{\sigma}{2}^2)t}{\sigma\sqrt{t}}
\end{equation}
<br>\begin{equation}
d_{2}=d_{1}-\sigma\sqrt{t}
\end{equation}



*Further contemplations:*

* The probability of price to be in-the-money for call/put options can be pictured as if seeing '*d*' landing in the right/left sides of the CDF (for call/put options, respectively).

* Probability of the put option to be in-the-money would be N(-d), representing the left tail of the CDF, which is the area from negative-infinity to zero minus *d*. The probability of a call option to be in-the-money would be 1-N(-d); the right-tail of the CDF.

* Price is not normally distributed. Instead, a lognormal distribution would better describe it. We still want to associate probability of price reaching X with the normal distribution, so we need to make the adjustments to *d*. Also, we will want to risk-adjust *d* according to *r*


##**The Greeks**
* Delta is the change of option's theoretical value with respect to price of the underlying asset
* Gamma is the change of Delta with respect to price of the underlying asset
* Theta is time decay, options from buyer's perspective, loses value as time approaches the maturity date. 
* Vega is the sensitivity of the option's theoretical value to a change in volatility 
* Rho, being the least popular Greek measurement among the above, is the sensitivity of the option’s theoretical value with respect to change of interest rate. Usually interest rate does not vary during the life of the option, and if it does, the changes are in small increments, thus the effect gets overshadowed by other Greek measurements we have introduced above. 

<br>

---
*References:*

[1] Natenberg, Sheldon. <i>Chapter 18, Option Volatility and Pricing, Second Edition</i>. McGraw-Hill Edu., 2015.

# **Implementation:**

In [3]:
class BSM:
  
  def __init__(self, s,t,r,sigma):

    # Fixed variables, will not change as we iterate through the option chain
    self.s = s
    self.t = t
    self.r = r
    self.sigma = sigma

    # variables that will change as we iterate through the option chain
    self.x, self.d1, self.d2 = None, None, None
    self.tv_call, self.delta_call , self.gamma_call, self.vega_call, self.theta_call, self.rho_call = None, None, None, None, None, None
    self.tv_put, self.delta_put, self.gamma_put, self.vega_put, self.theta_put, self.rho_put = None, None, None, None, None, None

  def _recalc(self):
    # _recalc() needs to be called everytime when an input variable is changed
    _a = math.log(self.s/ self.x)
    _b = (self.r+self.sigma**2/2)*self.t
    self.d1 = (_a+_b)/self.sigma*math.sqrt(self.t)
    self.d2 = self.d1 - self.sigma * math.sqrt(self.t)
    
    # Call: TV, delta, gamma, vega, theta, rho
    self.tv_call = self.s * norm.cdf(self.d1) - self.x*math.exp(-self.r*self.t)*norm.cdf(self.d2)
    self.delta_call = norm.cdf(self.d1)
    self.gamma_call = norm.pdf(self.d1)/(self.s*self.sigma*math.sqrt(self.t))
    self.vega_call = 0.01*(self.s*norm.pdf(self.d1)*math.sqrt(self.t))
    self.theta_call = 0.01*(-(self.s*norm.pdf(self.d1)*self.sigma)/(2*math.sqrt(self.t)) - self.r*self.x*math.exp(-self.r*self.t)*norm.cdf(self.d2))
    self.rho_call = 0.01*(self.x*self.t*math.exp(-self.r*self.t)*norm.cdf(self.d2))
    
    # Put: TV, delta, gamma, vega, theta, rho
    self.tv_put = self.x * math.exp(-self.r*self.t)-self.s+self.tv_call
    self.delta_put = -norm.cdf(-self.d1)
    self.gamma_put = norm.pdf(self.d1)/(self.s*self.sigma*math.sqrt(self.t))
    self.vega_put = 0.01*(self.s*norm.pdf(self.d1)*math.sqrt(self.t))
    self.theta_put = 0.01*(-(self.s*norm.pdf(self.d1)*self.sigma)/(2*math.sqrt(self.t)) + self.r*self.x*math.exp(-self.r*self.t)*norm.cdf(-self.d2))
    self.rho_put = 0.01*(-self.x*self.t*math.exp(-self.r*self.t)*norm.cdf(-self.d2))

  def set_x(self, x):
    self.x = x
    self._recalc()


####Validation:


Comparing results with @YuChenAmberLu's implementation [<link\>](https://github.com/YuChenAmberLu/Options-Calculator)

Expecting the results to be identical, if differences (rounded by 5 decimal places) match, then we will be satisfied. 




In [4]:
test_model = BSM(s=100,t=0.128767,r=0.2,sigma=0.2)
test_model.set_x(100)
assert (test_model.tv_call      -4.112199).round(5)==0
assert (test_model.delta_call   -.520269).round(5)==0
assert (test_model.gamma_call   -.055516).round(5)==0
assert (test_model.vega_call    -.142972).round(5)==0
assert (test_model.theta_call-  -.206861).round(5)==0
assert (test_model.rho_call     -.061698).round(5)==0
assert (test_model.tv_put       -1.569736).round(5)==0
assert (test_model.delta_put-   -.479731).round(5)==0
assert (test_model.gamma_put    -.055516).round(5)==0
assert (test_model.vega_put     -.142972).round(5)==0
assert (test_model.theta_put-   -.011946).round(5)==0
assert (test_model.rho_put-     -.063795).round(5)==0

#Application:
😎 Setting up our model:

In [5]:
# Manual inputs:
symbol            = "TSLA" 
riskfree_rate     = .0512 
sigma             = 0.7 
date_today        = date(2023,2,21) 
date_expire       = date(2023,3,10) 

In [6]:
# Confirming that the expiration date is valid.
exp_dates = options.get_expiration_dates(symbol)
date_expire_str = date_expire.strftime("%B %d, %Y") 
assert date_expire_str in exp_dates
exp_dates

['February 24, 2023',
 'March 3, 2023',
 'March 10, 2023',
 'March 17, 2023',
 'March 24, 2023',
 'March 31, 2023',
 'April 21, 2023',
 'May 19, 2023',
 'June 16, 2023',
 'July 21, 2023',
 'September 15, 2023',
 'December 15, 2023',
 'January 19, 2024',
 'March 15, 2024',
 'June 21, 2024',
 'September 20, 2024',
 'January 17, 2025',
 'June 20, 2025']

In [7]:
ticker_yahoo = yf.Ticker(symbol)
data = ticker_yahoo.history()
crnt_price = data['Close'].iloc[-1]
crnt_price

197.3699951171875

In [8]:
NAME_STRIKE  = "Strike"
NAME_QUOTE   = "Quote"
NAME_TV      = "Th.Value"
NAME_TVDIFF  = "Th.Edge"
NAME_DELTA   = "Δ"
NAME_GAMMA   = "𝚪"
NAME_VEGA    = "V"
NAME_THETA   = "Θ"
NAME_RHO     = "⍴"

FORMAT_COLS = [NAME_QUOTE, NAME_TV, NAME_TVDIFF, NAME_DELTA, NAME_GAMMA, NAME_VEGA, NAME_THETA, NAME_RHO]

def get_df(chain, option_type, date_expire_str, verbose=False):
  ''' @param Object chain : from options.get_options_chain(symbol, date_expire_str)["option_type"] 
      @param string option_type : either "calls" or "puts"
      @param string date_expire_str : element of options.get_expiration_dates(symbol)
  '''
  l_strike, l_quote, l_tv, l_diff, l_delta, l_gamma, l_vega, l_theta, l_rho, l_index_names = [], [], [], [], [], [], [], [], [], []

  for i in range(len(chain)):
    x = chain["Strike"][i]
    model.set_x(x)    
    
    tv, gamma, theta, vega, theta, rho = None, None, None, None, None, None
    if option_type == "calls":
      tv = model.tv_call
      delta, gamma, vega, theta, rho = model.delta_call, model.gamma_call, model.vega_call, model.theta_call, model.rho_call
    elif option_type == "puts":
      tv = model.tv_put
      delta, gamma, vega, theta, rho = model.delta_put, model.gamma_put, model.vega_put, model.theta_put, model.rho_put
    
    if tv > 0:
      q = chain["Last Price"][i]
      _diff = (tv-q).round(2)
      if verbose:
        print("{}: strike= {}\tquote= {}\tTV= {};\tdiff.= {}".format(option_type, x, q, tv.round(2), _diff ))

      l_index_names.append("{}_{}_{}".format(date_expire_str, x, option_type))
      l_delta.append(delta.round(4))
      l_gamma.append(gamma.round(4))
      l_theta.append(theta.round(4))
      l_vega.append(vega.round(4))
      l_rho.append(rho.round(4))
      l_tv.append(tv.round(2))
      l_diff.append(_diff)
      l_strike.append(x)
      l_quote.append(q)

  return pd.DataFrame({ NAME_STRIKE:l_strike, NAME_QUOTE:l_quote, NAME_TV:l_tv, NAME_TVDIFF:l_diff, \
                       NAME_DELTA:l_delta, NAME_GAMMA:l_gamma, NAME_VEGA:l_vega, NAME_THETA:l_theta, \
                       NAME_RHO:l_rho }, columns=FORMAT_COLS, index=l_index_names)

---
##Option Chain Analysis:

In [9]:
chain_call = options.get_options_chain(symbol, date_expire_str)["calls"]
chain_put = options.get_options_chain(symbol, date_expire_str)["puts"]

In [10]:
model = BSM(s     =crnt_price,
            t     =(date_expire-date_today).days/365,
            r     =riskfree_rate,
            sigma = sigma )

### Calls:

In [11]:
df_call = get_df(chain_call, "calls", date_expire_str)
print("{} [{}]\n".format(symbol, round(crnt_price,2)))
df_call

TSLA [197.37]



,Quote,Th.Value,Th.Edge,Δ,𝚪,V,Θ,⍴
"March 10, 2023_15.0_calls",196.16,144.39,-51.77,0.7878,0.0097,0.1235,-0.9339,0.0052
"March 10, 2023_20.0_calls",191.37,136.03,-55.34,0.7612,0.0104,0.1321,-0.9997,0.0066
"March 10, 2023_30.0_calls",183.58,122.27,-61.31,0.7207,0.0113,0.1432,-1.0863,0.0093
"March 10, 2023_40.0_calls",152.10,110.88,-41.22,0.6902,0.0118,0.1502,-1.1419,0.0118
"March 10, 2023_50.0_calls",144.38,100.98,-43.40,0.6655,0.0122,0.1551,-1.1810,0.0141
...,...,...,...,...,...,...,...,...
"March 10, 2023_215.0_calls",7.20,4.45,-2.75,0.4912,0.0134,0.1699,-1.3240,0.0431
"March 10, 2023_217.5_calls",6.48,3.39,-3.09,0.4898,0.0134,0.1699,-1.3243,0.0434
"March 10, 2023_220.0_calls",5.80,2.35,-3.45,0.4883,0.0134,0.1699,-1.3246,0.0438
"March 10, 2023_222.5_calls",5.25,1.31,-3.94,0.4870,0.0134,0.1698,-1.3248,0.0442


### Puts:

In [12]:
df_put = get_df(chain_put, "puts", date_expire_str)
print("{} [{}]\n".format(symbol, round(crnt_price,2)))
df_put

TSLA [197.37]



,Quote,Th.Value,Th.Edge,Δ,𝚪,V,Θ,⍴
"March 10, 2023_177.5_puts",5.60,0.81,-4.79,-0.4853,0.0134,0.1698,-1.2266,-0.0450
"March 10, 2023_180.0_puts",6.40,2.13,-4.27,-0.4870,0.0134,0.1698,-1.2260,-0.0458
"March 10, 2023_182.5_puts",7.05,3.47,-3.58,-0.4887,0.0134,0.1699,-1.2253,-0.0465
"March 10, 2023_185.0_puts",7.96,4.82,-3.14,-0.4903,0.0134,0.1699,-1.2246,-0.0473
"March 10, 2023_187.5_puts",8.77,6.17,-2.60,-0.4920,0.0134,0.1699,-1.2238,-0.0481
"March 10, 2023_190.0_puts",9.94,7.53,-2.41,-0.4936,0.0134,0.1699,-1.2231,-0.0489
"March 10, 2023_192.5_puts",10.90,8.90,-2.00,-0.4952,0.0134,0.1699,-1.2223,-0.0497
"March 10, 2023_195.0_puts",12.25,10.28,-1.97,-0.4968,0.0134,0.1699,-1.2215,-0.0505
"March 10, 2023_197.5_puts",13.48,11.66,-1.82,-0.4984,0.0134,0.1699,-1.2206,-0.0512
"March 10, 2023_200.0_puts",14.80,13.06,-1.74,-0.4999,0.0134,0.1699,-1.2198,-0.0520
